In [73]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from itertools import product


def prediction(symptomlist):

    #import data
    disease_data = pd.read_csv('/Users/ckelv/Downloads/41467_2014_BFncomms5212_MOESM1043_ESM.txt', sep='\t')
    symptom_data = pd.read_csv('/Users/ckelv/Downloads/41467_2014_BFncomms5212_MOESM1044_ESM.txt', sep='\t')
    connections_data = pd.read_csv('/Users/ckelv/Downloads/41467_2014_BFncomms5212_MOESM1045_ESM.txt', sep='\t')


    #filter data to only have connections with more than 500 occurrences
    connections_data = connections_data.loc[connections_data['PubMed occurrence'] >= 500]


    #add suffixes to distinguish diseases and symptoms
    disease_data['MeSH Disease Term'] = disease_data['MeSH Disease Term'].apply(lambda x: x + '_d')
    symptom_data['MeSH Symptom Term'] = symptom_data['MeSH Symptom Term'].apply(lambda x: x + '_s')
    connections_data['MeSH Disease Term'] = connections_data['MeSH Disease Term'].apply(lambda x: x + '_d')
    connections_data['MeSH Symptom Term'] = connections_data['MeSH Symptom Term'].apply(lambda x: x + '_s')

    #initialize network
    model = BayesianNetwork()

    #add nodes for diseases and symptoms
    diseases = disease_data['MeSH Disease Term'].unique()
    symptoms = symptom_data['MeSH Symptom Term'].unique()

    model.add_nodes_from(diseases)
    model.add_nodes_from(symptoms)

    #add edges based on cooccurrence data
    for index, row in connections_data.iterrows():
        disease = row['MeSH Disease Term']
        symptom = row['MeSH Symptom Term']
        if disease != symptom:
            model.add_edge(disease, symptom)

    #gather info for prior probabilities and coocurrence probabilities
    total_disease_occurrences = disease_data['PubMed occurrence'].sum()
    disease_prior_probs = (disease_data.set_index('MeSH Disease Term')['PubMed occurrence'] / total_disease_occurrences).to_dict()
    total_cooccurrences = connections_data["PubMed occurrence"].sum()
    total_symptom_occurrences = symptom_data['PubMed occurrence'].sum()

    #initialize list of cpds to be added
    cpds = []
    
    #add cpds of diseases (just the prior probs)
    for disease, prob in disease_prior_probs.items():
        cpd = TabularCPD(variable=disease, variable_card=2, values=[[1 - prob], [prob]])
        cpds.append(cpd)

    #create cpds for each symptom
    for symptom in symptoms:
        
        #create the small cpd table of just one disease and the symptom
        parents = model.get_parents(node = symptom)
        single_cpd = {}
        #single_cpd for a specific parent disease takes form: [(S=True & D=False), (S=True & D=True)]
        for parent in parents:
            #probs calculated as S=True & D=False -> (#occurences of symptom - #cooccurrences of symptom and disease)/#total symptom occurrences
            #S=True & D=True -> prob of cooccurrence/prob of disease

            single_cpd[parent] = [(symptom_data[symptom_data["MeSH Symptom Term"] == symptom]["PubMed occurrence"].iloc[0] -
                                    connections_data[(connections_data["MeSH Symptom Term"] == symptom) &
                                    (connections_data["MeSH Disease Term"] == parent)]["PubMed occurrence"].iloc[0]) /
                                    total_symptom_occurrences,
                                    (connections_data[(connections_data["MeSH Symptom Term"] == symptom) &
                                    (connections_data["MeSH Disease Term"] == parent)]["PubMed occurrence"].iloc[0]/total_cooccurrences) /
                                    (disease_data[disease_data["MeSH Disease Term"] == parent]["PubMed occurrence"].iloc[0]/total_disease_occurrences)]
        
        #initialize the row of values for when S=True in the final cpd
        true_cpd = []
        #iterate thru all combinations of parent occurrence and abscence to find the row of values when symptom is true
        for combo in product([0,1], repeat = len(parents)):
            total = 1
            for i in range(len(parents)):
                total *= single_cpd[parents[i]][combo[i]]
            true_cpd.append(total)
        #calculate row of values when symptom is false as 1 - true
        false_cpd = [1 - num for num in true_cpd]
        
        #add cpd to included
        cpd = TabularCPD(variable=symptom, variable_card=2,
                                 values=[false_cpd, true_cpd],
                                 evidence=parents,
                                 evidence_card=[2] * len(parents))
        cpds.append(cpd)

    #add cpds to model and validate
    model.add_cpds(*cpds)
    assert model.check_model(), "Model is not valid"

    #add _s to each symptom given
    symptomlist = [item + '_s' for item in symptomlist]
    
    #initialize inference object
    infer = VariableElimination(model)
    
    #create evidence dict as exhibited symptoms: 1
    evidence = {symptom: 1 for symptom in symptomlist}

    #run inference query on every disease, saving results in a dictionary of disease and confidence it is exhibited
    results = {}
    for disease in diseases:
        results[disease] = infer.query(variables = [disease], evidence = evidence, show_progress = False).values[1]
        #print(disease + ": " + str(results[disease]))

    #top_3_keys = [k for k, v in sorted(results.items(), key=lambda item: item[1], reverse=True)[:3]]
    #print(top_3_keys)
    
    #predicted disease with highest confidence
    predicted_disease = max(results, key=results.get)
    #output statement
    print(f"Using {len(connections_data)} entries, symptoms: {symptomlist} suggest {predicted_disease[:-2]} with confidence: {results[predicted_disease]}.")

In [74]:
def get_symptoms():
    symptoms = []
    print("Enter your symptoms one by one. Type 'done' when you are finished.")

    while True:
        symptom = input("Enter a symptom: ")
        if symptom.lower() == 'done':
            break
        symptoms.append(symptom)

    return symptoms

symptoms = get_symptoms()
prediction(symptoms)


Enter your symptoms one by one. Type 'done' when you are finished.
Enter a symptom: Angina Pectoris
Enter a symptom: done
Using 298 entries, symptoms: ['Angina Pectoris_s'] suggest Angina Pectoris with confidence: 0.7109990815261708.
